### *Assignment 2*: Group Recommendations

In [6]:
from group_recommendation import GroupRecommendation
import pandas as pd

userGroup = [1, 50, 600]

**(a)** For producing group recommendation, we will use the user-based collaborative
filtering approach as this implemented in Assignment 1. Specifically, for producing group
recommendations, we will first compute the movies recommendations for each user in
the group, and then we will aggregate the lists of the individual users, so as to produce a
single list of movies for the group.

You will implement two well established aggregation methods for producing the group
recommendations.

The first aggregation approach is the average method. The main idea behind this
approach is that all members are considered equals. So, the rating of an item for a group
of users will be given be averaging the scores of an item across all group members.
The second aggregation method is the least misery method, where one member can act
as a veto for the rest of the group. In this case, the rating of an item for a group of users is
computed as the minimum score assigned to that item in all group members
recommendations.

Produce a group of 3 users, and for this group, show the top-10 recommendations, i.e.,
the 10 movies with the highest prediction scores that: 
1. the average method suggests,
2. the least misery method suggests. 


In [16]:
aggregation = GroupRecommendation.aggregate_users_recommendations(userGroup)
pd.DataFrame(aggregation.items(), columns=['movieId', 'predictions'])

,movieId,preditions
0,319,[6.769157088122605]
1,3567,[6.726379310344827]
2,555,[6.641379310344828]
3,913,[6.252742946708464]
4,55276,[6.252742946708464]
5,30803,"[6.225754310344827, 4.850855996068152]"
6,3972,"[6.223522167487685, 4.63778801843318, 4.468753..."
7,27611,[6.223522167487685]
8,5066,"[6.110281749369218, 4.524547600314714]"
9,42728,"[6.110281749369218, 4.524547600314714]"


In [9]:
aggregation = GroupRecommendation.average_aggregation(userGroup)
pd.DataFrame(aggregation, columns=['movieId', 'avg predicition'])

,movieId,avg predicition
0,319,6.769157
1,3567,6.726379
2,555,6.641379
3,913,6.252743
4,55276,6.252743
5,27611,6.223522
6,30803,5.538305
7,5066,5.317415
8,42728,5.317415
9,3972,5.110021


In [10]:
aggregation = GroupRecommendation.least_misery_aggregation(userGroup)
pd.DataFrame(aggregation, columns=['movieId', 'min predicition'])

,movieId,min predicition
0,319,6.769157
1,3567,6.726379
2,555,6.641379
3,913,6.252743
4,55276,6.252743
5,27611,6.223522
6,30803,4.850856
7,5014,4.841251
8,1204,4.710836
9,3469,4.562910
